In [1]:
import dataprep.connector as dbc
import urllib
table_name = "injury_information"
mshp_db_path = r"C:\Users\lgarzia\Documents\GitHub\explainability\local_data\data\mshp.sqlite" 
mshp_db_path = urllib.parse.quote(mshp_db_path)
mshp_db_conn = "sqlite://"+mshp_db_path
df = dbc.read_sql(mshp_db_conn, f"SELECT * FROM {table_name}")

In [2]:
from ydata_profiling import ProfileReport

Pandas backend loaded 1.5.3
Numpy backend loaded 1.23.5
Pyspark backend NOT loaded
Python backend loaded


In [3]:
profile = ProfileReport(df, title=f"Profiling Report for {table_name}")

In [4]:
profile.to_file(f"Profiling Report for {table_name}")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\lgarzia\Documents\GitHub\explainability\venv\lib\site-packages\ydata_profiling\profile_report.py:355: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df_g = df.groupby(['acc_uuid']).filter(lambda x: x['df_index'].count() > 1)
df_g.head(n=5)

,df_index,Veh. #,Name,Gender,Age,Injury Type,Safety Device,City/State,Involvement,Disposition,acc_uuid
4,0,1,"JUVENILE,",MALE,12,MODERATE,YES,"ST PETERS, MO",OCCUPANT,TRANSPORTED TO SSM HEALTH ST JOSEPH HOSPITAL L...,0f9c245e-1373-4c73-a443-b3623270e573
5,1,1,"JUVENILE,",MALE,15,MODERATE,YES,"ST PETERS, MO",DRIVER,TRANSPORTED TO SSM HEALTH ST JOSEPH HOSPITAL L...,0f9c245e-1373-4c73-a443-b3623270e573
6,2,1,"BISHOP, BRIAN R",MALE,48,SERIOUS,YES,"ST PETERS, MO",OCCUPANT,TRANSPORTED TO SSM HEALTH ST JOSEPH HOSPITAL L...,0f9c245e-1373-4c73-a443-b3623270e573
8,0,1,"PONTREMOLI, STEPHEN A",MALE,46,MODERATE,YES,"ST JAMES, MO",DRIVER,TRANSPORTED BY ST JAMES AMBULANCE TO PHELPS HE...,16fe17ef-6e0b-4192-825d-1640b814e785
9,1,2,"HAGGARD, DEBORAH A",FEMALE,61,MINOR,YES,"ST JOHN, MO",OCCUPANT,TRANSPORTED BY ST JAMES AMBULANCE TO PHELPS HE...,16fe17ef-6e0b-4192-825d-1640b814e785


In [15]:
sql = '''	select DISTINCT dcr.acc_rpt_num, dcr.acc_uuid, 
	coalesce(dcr.Name, ii.Name) Name,  
	coalesce(dcr."Personal Injury", ii."Injury Type") injury_type, 
	coalesce(dcr.Age, ii.Age) Age,
	coalesce(dcr."Person City/State", ii."City/State") city_state,
	dcr."Safety Device", dcr.Date, dcr.Time, dcr.Troop, dcr."Crash Location",
    ii.Gender,
	ii."Veh. #", 
	ii."Involvement",
	ii.Disposition
	from daily_crash_report dcr
	left outer join injury_information ii on dcr.acc_uuid = ii.acc_uuid  --13751, inner(10129)
	and dcr.Name = ii.Name 
	and dcr.Age = ii.Age 
	and dcr."Person City/State" = ii."City/State"
	and dcr."Personal Injury" = ii."Injury Type"
'''

In [16]:
df = dbc.read_sql(mshp_db_conn, sql)

In [17]:
profile = ProfileReport(df, title=f"Profiling Report for crs-injury-information")

In [18]:
profile.to_file(f"Profiling Report for crs-injury-information")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\lgarzia\Documents\GitHub\explainability\venv\lib\site-packages\ydata_profiling\profile_report.py:355: UserWarning: Extension  not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(


Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]